In [23]:

import requests
from bs4 import BeautifulSoup

import csv
import pandas as pd
import re
import math
url = "https://www.kabum.com.br/hardware/placa-de-video-vga"
base_url = "https://www.kabum.com.br/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
qtd_itens = soup.find('div', id='listingCount').get_text().strip()
index = qtd_itens.find(' ')
qtd = qtd_itens[:index]
ultima_pagina = math.ceil(int(qtd)/20)
lista_produtos = []




for i in range(1, ultima_pagina+1):
    url_pag = f'https://www.kabum.com.br/hardware/placa-de-video-vga?page_number={i}&page_size=20&facet_filters=&sort=most_searched'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')

    produtos = soup.find_all('div', class_='productCard')

    for produto in produtos:
         if produto.find('div', class_=re.compile('unavailablePricesCard')):
             continue

         link = base_url + produto.find('a', class_=re.compile('productLink')).get('href').strip()

         # Faz a requisição para a página do produto
         site_produto = requests.get(link, headers=headers)
         soup_produto = BeautifulSoup(site_produto.content, 'html.parser')
         infor = soup_produto.find('div', class_=re.compile('col-purchase'))
         #encontrar preços no pix
         preco_pix = None
         if infor:
             preco_pix_element = infor.find('h4', class_='finalPrice')
             if preco_pix_element:
                 preco_pix = preco_pix_element.get_text().strip()
         #encontrar nome   
         marca = None
         if infor:
             marca_element = infor.find('h1', class_='cMCMNo')
             if marca_element:
                 marca = marca_element.get_text().strip()

    
         #encontrar preços original
         preco_original = None
         if infor:
             preco_original_element = infor.find('span', class_='oldPrice')
             if preco_original_element:
                 preco_original = preco_original_element.get_text().strip()
          #encontrar preços no prazo
         regular_price_tag = None
         if infor:
             regular_price_element = infor.find('b', class_='regularPrice')
             if regular_price_element:
                 regular_price_tag = regular_price_element.get_text().strip()

         dados_produto = {
             'nome': marca,
             'preço pix': preco_pix,
             'preco a prazo': regular_price_tag,
             'preço original': preco_original,
             'link': link,
         }

         lista_produtos.append(dados_produto)

         print(marca, preco_pix, preco_original, link)

In [19]:
#csv
with open('PlacadeVideo.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['nome', 'preço pix', 'preco a prazo','preço original', 'link'])
    writer.writeheader()
    writer.writerows(lista_produtos)

In [20]:
df = pd.DataFrame(lista_produtos)

In [21]:
df

,Produtos,Preço Pix,Preços parcelado,Preco Original,Links
0,Placa de Vídeo RX 6750 XT Mech 2X 12G OC MSI A...,"R$ 2.699,99","R$ 3.176,46","R$ 3.411,75",https://www.kabum.com.br/produto/417853/placa-...
1,Placa de Vídeo RTX 3060 Asus Dual O12G V2 NVID...,"R$ 2.299,99","R$ 2.705,87","R$ 2.904,69",https://www.kabum.com.br/produto/164854/placa-...
2,Placa de Vídeo RX 6600 CLD 8G ASRock AMD Radeo...,"R$ 1.559,99","R$ 1.835,28","R$ 2.223,52",https://www.kabum.com.br/produto/235984/placa-...
3,Placa de Vídeo RTX 3050 XLR8 Gaming Revel Epic...,"R$ 1.739,99","R$ 2.047,05",,https://www.kabum.com.br/produto/367414/placa-...
4,"Placa de Video PNY NVIDIA Geforce RTX 3060, 12...","R$ 2.378,99","R$ 2.643,32",,https://www.kabum.com.br/produto/383616/placa-...
...,...,...,...,...,...
625,Placa de Vídeo GTX 1630 EX Galax Nvidia GeForc...,"R$ 839,99","R$ 988,22","R$ 1.047,05",https://www.kabum.com.br/produto/380802/placa-...
626,Placa de Vídeo RTX 3060 Ti Twin Edge Zotac Gam...,"R$ 2.999,99","R$ 3.529,40",,https://www.kabum.com.br/produto/403365/placa-...
627,Placa de Vídeo RTX 3060 Ti 1-Click OC Galax NV...,"R$ 2.656,99","R$ 2.952,21","R$ 3.111,10",https://www.kabum.com.br/produto/191789/placa-...
628,Placa de Vídeo RTX 3060 1-Click OC Galax GeFor...,"R$ 2.378,99","R$ 2.643,32","R$ 2.811,10",https://www.kabum.com.br/produto/150657/placa-...
